### Import Relevant Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
sns.set()
pd.options.display.max_columns = 10
pd.set_option('display.max_rows',10)

### Import the data

### Data from fbref.com

In [2]:
defensiveAction = pd.read_csv("DefensiveAction.csv")
shooting = pd.read_csv("Shooting.csv")
gca = pd.read_csv("GCA.csv")
passing = pd.read_csv("Passing.csv")
goalkeeper = pd.read_csv("gk.csv")
advGoalkeeper = pd.read_csv("AdvancedGk.csv")
msc = pd.read_csv("Msc.csv")

### Dropping Unnecessary Columns

In [3]:
defensiveAction = defensiveAction.drop(['Matches', '-9999','Rk'],axis=1)
shooting = shooting.drop(['Matches', '-9999','Rk'],axis=1)
gca = gca.drop(['Matches', '-9999','Rk'],axis=1)
passing = passing.drop(['Matches', '-9999','Rk'],axis=1)
goalkeeper = goalkeeper.drop(['Matches', '-9999','Rk'],axis=1)
advGoalkeeper = advGoalkeeper.drop(['Matches', '-9999','Rk'],axis=1)
msc = msc.drop(['Matches','-9999','CrdY', 'CrdR', '2CrdY', 'Fls', 'Fld', 'Off', 'Crs', 'Int', 'TklW',
       'PKwon', 'PKcon', 'OG','Rk'],axis=1)

### Joining Goalkeeper Data

In [4]:
merged_data_gk = pd.merge(goalkeeper,advGoalkeeper,on=['Player','Nation','Pos','Squad','Comp','Age','Born','90s','GA','PKA'])

In [5]:
merged_data_gk = merged_data_gk.drop(['Nation', 'Squad','Comp','Pos'],axis=1)

### Joining Position Player Data

In [6]:
merged_data_pos = pd.merge(defensiveAction,shooting,on=['Player','Nation','Pos','Squad','Comp','Age','Born','90s'])
merged_data_pos = pd.merge(merged_data_pos,gca,on=['Player','Nation','Pos','Squad','Comp','Age','Born','90s'])
merged_data_pos = pd.merge(merged_data_pos,passing,on=['Player','Nation','Pos','Squad','Comp','Age','Born','90s'])
merged_data_pos = pd.merge(merged_data_pos,msc,on=['Player','Nation','Pos','Squad','Comp','Age','Born','90s'])

In [7]:
merged_data_pos = merged_data_pos.drop(['Squad','Born','Nation'],axis=1)

In [8]:
merged_data_pos

,Player,Pos,Comp,Age,90s,...,Prog,Recov,Won,Lost,Won%
0,Max Aarons,DF,eng Premier League,21.0,32.0,...,94,177,15,51,22.7
1,Yunis Abdelhamid,DF,fr Ligue 1,33.0,33.1,...,90,224,67,45,59.8
2,Salis Abdul Samed,MF,fr Ligue 1,21.0,27.4,...,81,240,24,24,50.0
3,Laurent Abergel,MF,fr Ligue 1,28.0,32.8,...,137,291,14,14,50.0
4,Charles Abi,FW,fr Ligue 1,21.0,0.5,...,0,2,1,0,100.0
...,...,...,...,...,...,...,...,...,...,...,...
2911,Martín Zubimendi,MF,es La Liga,22.0,28.8,...,106,271,76,40,65.5
2912,Szymon Żurkowski,MF,it Serie A,23.0,25.6,...,68,206,26,40,39.4
2913,Martin Ødegaard,MF,eng Premier League,22.0,30.9,...,158,230,17,25,40.5
2914,Milan Đurić,FW,it Serie A,31.0,24.1,...,30,78,298,116,72.0


### Dropping non-qualified players

In [9]:
merged_data_pos_qualified = merged_data_pos[merged_data_pos['Pos']!= 'GK']
merged_data_pos_qualified = merged_data_pos_qualified[merged_data_pos_qualified['90s']>= 19]

In [10]:
merged_data_pos_qualified.isna().sum()

Player    0
Pos       0
Comp      0
Age       0
90s       0
         ..
Prog      0
Recov     0
Won       0
Lost      0
Won%      0
Length: 87, dtype: int64

Column that has null value
- TKL%
- G/SoT

In [11]:
merged_data_pos_qualified[merged_data_pos_qualified['G/SoT'].isna()]

,Player,Pos,Comp,Age,90s,...,Prog,Recov,Won,Lost,Won%
74,Yeray Álvarez,DF,es La Liga,26.0,20.7,...,61,253,70,46,60.3
108,Dennis Appiah,DF,fr Ligue 1,29.0,23.3,...,75,166,36,44,45.0
384,Cédric Brunner,DF,de Bundesliga,27.0,24.3,...,76,240,57,39,59.4
464,Andrea Carboni,DF,it Serie A,20.0,24.6,...,81,223,59,45,56.7
476,Nicolò Casale,DF,it Serie A,23.0,28.7,...,73,292,85,63,57.4
...,...,...,...,...,...,...,...,...,...,...,...
2352,Mamadou Sakho,DF,fr Ligue 1,31.0,28.6,...,91,267,63,39,61.8
2576,Stefano Sturaro,MF,it Serie A,28.0,19.6,...,50,204,54,59,47.8
2614,Charlie Taylor,DF,eng Premier League,27.0,30.3,...,79,275,66,34,66.0
2638,Luke Thomas,DF,eng Premier League,20.0,21.2,...,73,192,30,39,43.5


- Null TKL% and G/SoT data is not a missing value, that player really have 0 in TKL% (because they didn' attempt any tackle), or 0 in G/SoT (Didn't have any shot on target), so i'll fill it with 0

In [12]:
merged_data_pos_qualified['Tkl%'] = merged_data_pos_qualified['Tkl%'].fillna(value=0)
merged_data_pos_qualified['G/SoT'] = merged_data_pos_qualified['G/SoT'].fillna(value=0)

In [13]:
merged_data_pos_qualified[merged_data_pos_qualified['Tkl%'].isna()]

,Player,Pos,Comp,Age,90s,...,Prog,Recov,Won,Lost,Won%


In [14]:
merged_data_pos_qualified[merged_data_pos_qualified['G/SoT'].isna()]

,Player,Pos,Comp,Age,90s,...,Prog,Recov,Won,Lost,Won%


Column that has similar name, and it's meaning 
- Tkl -> Number of Players tackled
- Tkl.1 -> Number of Dribblers Tackled
- Tkl%.1 -> Number of tackles win divided by total tackle attempts
- Def 3rd, Mid 3rd , Att 3rd -> Tackles
- Def 3rd.1 , Mid 3rd.1, Att 3rd.1 -> Applying pressures
- Sh_x -> shot blocked by standing in it's path
- Sh_y -> shot total (not including penalty kicks)
- Pass Live, Pass Dead -> Live ball and dead ball passes that lead to a shot attempt
- Drib -> Succesful dribble that lead to shot attempt
- Sh -> Shot that leads to another shot attempt
- Fld -> Fouls drawn that lead to shot attempt
- Def -> Defensive Action that lead to shot attempt
- Pass Live.1 , Pass Dead.1 -> Live ball and dead ball passes that lead to a goal
- Drib.1 -> Succesful dribble that lead to goal
- Sh.1 -> Shot that leads to another goal
- Fld.1 -> Fouls drawn that lead to goal
- Def.1 -> Defensive Action that lead to goal
- Att_x -> Number of dribbled passed plus number of tackles (Tkl.1 + Past)
- Cmp,Att_y, Cmp% ->Total Passes Completed, Attempted, and Total Passes Completed Percentage
- Cmp,Att,Cmp%.1 -> Short Passes Completed, Attempted, and Total Passes Completed Percentage (between 5 and 15 yards)
- Cmp,Att,Cmp%.2 -> Medium Passes Completed, Attempted, and Total Passes Completed Percentage (between 15 and 30 yards)
- Cmp,Att,Cmp%.3 -> Long Passes Completed, Attempted, and Total Passes Completed Percentage (longer than 30 yards)

In [15]:
merged_data_pos_qualified.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 879 entries, 0 to 2914
Data columns (total 87 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Player      879 non-null    object 
 1   Pos         879 non-null    object 
 2   Comp        879 non-null    object 
 3   Age         879 non-null    float64
 4   90s         879 non-null    float64
 5   Tkl         879 non-null    int64  
 6   TklW        879 non-null    int64  
 7   Def 3rd     879 non-null    int64  
 8   Mid 3rd     879 non-null    int64  
 9   Att 3rd     879 non-null    int64  
 10  Tkl.1       879 non-null    int64  
 11  Att_x       879 non-null    int64  
 12  Tkl%        879 non-null    float64
 13  Past        879 non-null    int64  
 14  Press       879 non-null    int64  
 15  Succ        879 non-null    int64  
 16  %           879 non-null    float64
 17  Def 3rd.1   879 non-null    int64  
 18  Mid 3rd.1   879 non-null    int64  
 19  Att 3rd.1   879 non-null    

### Column Groups

### Defensive Action
- Defense : Tackles,vs Dribbles, Tackle%, Pressures, Blocks
- Vs Dribbles : Tkl.1, Att.x, Tkl%, Past
- Pressures : Press, Succ, %, Def 3rd.1, Mid 3rd.1, Att 3rd.1
- Blocks : Blocks, Sh_x, ShSv, Pass
- Etc :  Int, Tkl+Int, Clr, Err

### Shooting
- Standard :Gls, Sh_y, SoT, SoT%, Sh/90, SoT/90, G/Sh, G/SoT, Dist, Fk, Pk, PKatt
- Expected : xG, npxG, npxG/Sh, G-xG, np:G-xG

### GCA (Goal Creation Action)
- SCA (Shot Creating Action) : SCA, SCA90
- SCA Types : PassLive, PassDead, Drib, Sh, Fld, Def
- GCA : GCA, GCA90
- GCA Types : PassLive.1, PassDead.1, Drib.1, Sh.1, Fld.1, Def.1

### Passing
- Total : Cmp, Att_y, Cmp%, TotDist, PrgDist
- Short : Cmp.1, Att.1, Cmp%.1
- Medium : Cmp.2, Att.2, Cmp%.2
- Long : Cmp.3, Att.3, Cmp%.3
- Etc : Ast, xA, A-xA, KP, 1/3, PPA, CrsPA, Prog

In [16]:
merged_data_pos_qualified.describe()

,Age,90s,Tkl,TklW,Def 3rd,...,Prog,Recov,Won,Lost,Won%
count,879.000000,879.000000,879.000000,879.000000,879.000000,...,879.000000,879.000000,879.000000,879.000000,879.000000
mean,26.053470,26.293402,45.607509,27.895336,22.233220,...,86.643914,209.902162,45.713311,43.300341,48.341183
std,3.868347,4.738237,21.203204,13.657225,13.718085,...,43.001980,76.772506,35.275611,27.594417,14.562125
min,16.000000,19.000000,2.000000,2.000000,0.000000,...,9.000000,43.000000,1.000000,2.000000,6.100000
25%,23.000000,22.250000,30.500000,18.000000,11.000000,...,55.500000,153.500000,19.500000,26.000000,38.200000
50%,26.000000,25.800000,44.000000,26.000000,21.000000,...,79.000000,205.000000,36.000000,37.000000,48.800000
75%,29.000000,30.100000,59.000000,36.000000,31.000000,...,109.000000,260.000000,63.000000,51.000000,59.600000
max,39.000000,38.000000,130.000000,80.000000,74.000000,...,278.000000,459.000000,314.000000,225.000000,81.800000


In [17]:
merged_data_pos_qualified['Tkl%.1'] = merged_data_pos_qualified['TklW']/merged_data_pos_qualified['Tkl']*100
merged_data_pos_qualified['Tkl%.1'] = merged_data_pos_qualified['Tkl%.1'].round(decimals=1)

In [18]:
merged_data_pos_qualified

,Player,Pos,Comp,Age,90s,...,Recov,Won,Lost,Won%,Tkl%.1
0,Max Aarons,DF,eng Premier League,21.0,32.0,...,177,15,51,22.7,53.6
1,Yunis Abdelhamid,DF,fr Ligue 1,33.0,33.1,...,224,67,45,59.8,74.2
2,Salis Abdul Samed,MF,fr Ligue 1,21.0,27.4,...,240,24,24,50.0,61.8
3,Laurent Abergel,MF,fr Ligue 1,28.0,32.8,...,291,14,14,50.0,62.4
7,Tammy Abraham,FW,it Serie A,23.0,34.3,...,126,82,99,45.3,64.7
...,...,...,...,...,...,...,...,...,...,...,...
2908,Kurt Zouma,DF,eng Premier League,26.0,23.1,...,164,55,25,68.8,45.5
2911,Martín Zubimendi,MF,es La Liga,22.0,28.8,...,271,76,40,65.5,52.5
2912,Szymon Żurkowski,MF,it Serie A,23.0,25.6,...,206,26,40,39.4,69.0
2913,Martin Ødegaard,MF,eng Premier League,22.0,30.9,...,230,17,25,40.5,58.1


### Renaming the Column, so it'll be easier to understand

In [19]:
merged_data_pos_qualified = merged_data_pos_qualified.rename(columns={'Player' : 'Name',
                                                                      'Comp' : 'League',
                                                                      'Tkl' : 'TklAtt',
                                                                      'TklW' : 'TklWon',
                                                                      'Tkl%.1' : 'Tkl_Percentage%',
                                                                      'Def 3rd' : 'Def 3rd (Tkl)',
                                                                      'Mid 3rd' : 'Mid 3rd (Tkl)',
                                                                      'Att 3rd' : 'Att 3rd (Tkl)',
                                                                      'Tkl.1' : 'TklvsDrib',
                                                                      'Att_x' : 'TklAttvsDrib',
                                                                      'Tkl%' : 'Tkl%vsDrib',
                                                                      'Def 3rd.1' : 'Def 3rd (Pres)',
                                                                      'Mid 3rd.1' : 'Mid 3rd (Pres)',
                                                                      'Att 3rd.1' : 'Att 3rd (Pres)',
                                                                      '%' : 'Pres%',
                                                                      'Sh_x' : 'ShotBlocked',
                                                                      'Sh_y' : 'ShotsTotal',
                                                                      'PassLive' : 'PassLive(SCA)',
                                                                      'PassDead' : 'PassDead(SCA)',
                                                                      'Drib' : 'Drib(SCA)',
                                                                      'Sh' : 'Sh(SCA)',
                                                                      'Fld' : 'Fld(SCA)',
                                                                      'Def' : 'Def(SCA)',
                                                                      'PassLive.1' : 'PassLive(GCA)',
                                                                      'PassDead.1' : 'PassDead(GCA)',
                                                                      'Drib.1' : 'Drib(GCA)',
                                                                      'Sh.1' : 'Sh(GCA)',
                                                                      'Fld.1' : 'Fld(GCA)',
                                                                      'Def.1' : 'Def(GCA)',
                                                                      'Cmp' : 'PassCmp',
                                                                      'Att_y' : 'PassAtt',
                                                                      'Cmp.1' : 'PassCmp(Short)',
                                                                      'Att.1' : 'PassAtt(Short)',
                                                                      'Cmp%.1' : 'Cmp%(Short)',
                                                                      'Cmp.2' : 'PassCmp(Medium)',
                                                                      'Att.2' : 'PassAtt(Medium)',
                                                                      'Cmp%.2' : 'Cmp%(Medium)',
                                                                      'Cmp.3' : 'PassCmp(Long)',
                                                                      'Att.3' : 'PassAtt(Long)',
                                                                      'Cmp%.3' : 'Cmp%(Long)',
                                                                      'Won' : 'HeadersWon',
                                                                      'Lost' : 'HeadersLost',
                                                                      'Won%' : 'HeadersWon%'})

### Calculatin per 90 Stats, and Headers Att

In [20]:
merged_data_pos_qualified['TklWon/90'] = merged_data_pos_qualified['TklWon'] / merged_data_pos_qualified['90s']
merged_data_pos_qualified['TklWon/90'] = merged_data_pos_qualified['TklWon/90'].round(decimals=1)
merged_data_pos_qualified['Succ/90'] = merged_data_pos_qualified['Succ'] / merged_data_pos_qualified['90s']
merged_data_pos_qualified['Succ/90'] = merged_data_pos_qualified['Succ/90'].round(decimals=1)
merged_data_pos_qualified['Blocks/90'] = merged_data_pos_qualified['Blocks'] / merged_data_pos_qualified['90s']
merged_data_pos_qualified['Blocks/90'] = merged_data_pos_qualified['Blocks/90'].round(decimals=1)
merged_data_pos_qualified['Int/90'] = merged_data_pos_qualified['Int'] / merged_data_pos_qualified['90s']
merged_data_pos_qualified['Int/90'] = merged_data_pos_qualified['Int/90'].round(decimals=1)
merged_data_pos_qualified['PassCmp/90'] = merged_data_pos_qualified['PassCmp'] / merged_data_pos_qualified['90s']
merged_data_pos_qualified['PassCmp/90'] = merged_data_pos_qualified['PassCmp/90'].round(decimals=1)
merged_data_pos_qualified['HeadersWon/90'] = merged_data_pos_qualified['HeadersWon'] / merged_data_pos_qualified['90s']
merged_data_pos_qualified['HeadersWon/90'] = merged_data_pos_qualified['HeadersWon/90'].round(decimals=1)
merged_data_pos_qualified['HeadersAtt'] = merged_data_pos_qualified['HeadersWon'] + merged_data_pos_qualified['HeadersLost']

### Rearranging Columns

In [21]:
cols = ['Name', 'Pos', 'League', 'Age', '90s', 'TklAtt', 'TklWon',
       'Tkl_Percentage%', 'TklWon/90', 'Def 3rd (Tkl)', 'Mid 3rd (Tkl)', 'Att 3rd (Tkl)',
       'TklvsDrib', 'TklAttvsDrib', 'Tkl%vsDrib', 'Past', 'Succ', 'Press',
       'Pres%', 'Succ/90', 'Def 3rd (Pres)', 'Mid 3rd (Pres)', 'Att 3rd (Pres)', 'Blocks','Blocks/90',
       'ShotBlocked', 'ShSv', 'Pass', 'Int', 'Int/90', 'Tkl+Int', 'Clr', 'Err', 'Gls',
       'ShotsTotal', 'SoT', 'SoT%', 'Sh/90', 'SoT/90', 'G/Sh', 'G/SoT', 'Dist',
       'FK', 'PK', 'PKatt', 'xG', 'npxG', 'npxG/Sh', 'G-xG', 'np:G-xG', 'SCA',
       'SCA90', 'PassLive(SCA)', 'PassDead(SCA)', 'Drib(SCA)', 'Sh(SCA)',
       'Fld(SCA)', 'Def(SCA)', 'GCA', 'GCA90', 'PassLive(GCA)',
       'PassDead(GCA)', 'Drib(GCA)', 'Sh(GCA)', 'Fld(GCA)', 'Def(GCA)',
       'PassCmp', 'PassAtt', 'Cmp%', 'PassCmp/90', 'TotDist', 'PrgDist', 'PassCmp(Short)',
       'PassAtt(Short)', 'Cmp%(Short)', 'PassCmp(Medium)', 'PassAtt(Medium)',
       'Cmp%(Medium)', 'PassCmp(Long)', 'PassAtt(Long)', 'Cmp%(Long)', 'Ast',
       'xA', 'A-xA', 'KP', '1/3', 'PPA', 'CrsPA', 'Prog', 'Recov',
       'HeadersWon', 'HeadersLost', 'HeadersAtt','HeadersWon/90', 'HeadersWon%']

In [22]:
merged_data_pos_qualified = merged_data_pos_qualified[cols]

In [23]:
merged_data_pos_qualified

,Name,Pos,League,Age,90s,...,HeadersWon,HeadersLost,HeadersAtt,HeadersWon/90,HeadersWon%
0,Max Aarons,DF,eng Premier League,21.0,32.0,...,15,51,66,0.5,22.7
1,Yunis Abdelhamid,DF,fr Ligue 1,33.0,33.1,...,67,45,112,2.0,59.8
2,Salis Abdul Samed,MF,fr Ligue 1,21.0,27.4,...,24,24,48,0.9,50.0
3,Laurent Abergel,MF,fr Ligue 1,28.0,32.8,...,14,14,28,0.4,50.0
7,Tammy Abraham,FW,it Serie A,23.0,34.3,...,82,99,181,2.4,45.3
...,...,...,...,...,...,...,...,...,...,...,...
2908,Kurt Zouma,DF,eng Premier League,26.0,23.1,...,55,25,80,2.4,68.8
2911,Martín Zubimendi,MF,es La Liga,22.0,28.8,...,76,40,116,2.6,65.5
2912,Szymon Żurkowski,MF,it Serie A,23.0,25.6,...,26,40,66,1.0,39.4
2913,Martin Ødegaard,MF,eng Premier League,22.0,30.9,...,17,25,42,0.6,40.5


### Changing League Name

In [24]:
merged_data_pos_qualified['League'] = merged_data_pos_qualified['League'].replace(['fr Ligue 1','eng Premier League',
                                                                                   'it Serie A','es La Liga','de Bundesliga'],
                                                                                 ['Ligue 1','Premier League','Serie A','La Liga','Bundesliga'])

In [25]:
merged_data_pos_qualified

,Name,Pos,League,Age,90s,...,HeadersWon,HeadersLost,HeadersAtt,HeadersWon/90,HeadersWon%
0,Max Aarons,DF,Premier League,21.0,32.0,...,15,51,66,0.5,22.7
1,Yunis Abdelhamid,DF,Ligue 1,33.0,33.1,...,67,45,112,2.0,59.8
2,Salis Abdul Samed,MF,Ligue 1,21.0,27.4,...,24,24,48,0.9,50.0
3,Laurent Abergel,MF,Ligue 1,28.0,32.8,...,14,14,28,0.4,50.0
7,Tammy Abraham,FW,Serie A,23.0,34.3,...,82,99,181,2.4,45.3
...,...,...,...,...,...,...,...,...,...,...,...
2908,Kurt Zouma,DF,Premier League,26.0,23.1,...,55,25,80,2.4,68.8
2911,Martín Zubimendi,MF,La Liga,22.0,28.8,...,76,40,116,2.6,65.5
2912,Szymon Żurkowski,MF,Serie A,23.0,25.6,...,26,40,66,1.0,39.4
2913,Martin Ødegaard,MF,Premier League,22.0,30.9,...,17,25,42,0.6,40.5


### Exporting the CSV

In [26]:
#merged_data_pos_qualified.to_csv('Position_Player.csv')